In [2]:
using Memoize
input = readlines("input.txt")
input[1] = replace(input[1], "Hit Points" => "HP") #doesnt compile without this, very important
hp = 100
input = [split(line, r":") for line in input]
input = Dict([(i[1], parse(Int, i[2])) for i in input])
input

Dict{SubString{String}, Int64} with 3 entries:
  "Armor"  => 2
  "HP"     => 109
  "Damage" => 8

In [3]:
shop = readlines("shop.txt")
shop

21-element Vector{String}:
 "Weapons:    Cost  Damage  Armor"
 "Dagger        8     4       0"
 "Shortsword   10     5       0"
 "Warhammer    25     6       0"
 "Longsword    40     7       0"
 "Greataxe     74     8       0"
 ""
 "Armor:      Cost  Damage  Armor"
 "Leather      13     0       1"
 "Chainmail    31     0       2"
 "Splintmail   53     0       3"
 "Bandedmail   75     0       4"
 "Platemail   102     0       5"
 ""
 "Rings:      Cost  Damage  Armor"
 "Damage +1    25     1       0"
 "Damage +2    50     2       0"
 "Damage +3   100     3       0"
 "Defense +1   20     0       1"
 "Defense +2   40     0       2"
 "Defense +3   80     0       3"

In [5]:
weapons = reduce(hcat, 
    [parse.(Int, split(m, r"   *")[2:end]) for m in shop[2:6]])
armors = reduce(hcat, 
    [parse.(Int, split(m, r"   *")[2:end]) for m in shop[9:13]])
armors = hcat(armors, [0,0,0])
rings = reduce(hcat, 
    [parse.(Int, split(m, r"   *")[2:end]) for m in shop[16:end]])
rings = hcat(rings, [0,0,0], [0,0,0])
shopp = hcat(weapons, armors, rings)
costs = shopp[1, :]
damage = shopp[2, :]
armor = shopp[3, :]
shopp

3×19 Matrix{Int64}:
 8  10  25  40  74  13  31  53  75  102  0  25  50  100  20  40  80  0  0
 4   5   6   7   8   0   0   0   0    0  0   1   2    3   0   0   0  0  0
 0   0   0   0   0   1   2   3   4    5  0   0   0    0   1   2   3  0  0

In [6]:
function ttk(damage)
    dmg = max(1, damage-input["Armor"])
    return ceil(input["HP"]/dmg)
end

function ttd(armor)
    bossdmg = max(1,input["Damage"]-armor)
    return ceil(hp/bossdmg)
end
@memoize function works(damage, armor)
    _ttk = ttk(damage)
    _ttd = ttd(armor)
    return _ttk <= _ttd, _ttk/_ttd
end

works(4,7)

(true, 0.55)

In [45]:
perms = reduce(hcat, sort(unique([w .+ a .+ r1 .+ r2 for (r2i, r2) in enumerate(eachcol(rings)) 
                                                     for (r1i, r1) in enumerate(eachcol(rings)) 
                                                     for a in eachcol(armors) 
                                                     for w in eachcol(weapons) if r1i != r2i]),
                        lt = (x,y) -> isless(x[1], y[1])))

3×616 Matrix{Int64}:
 8  10  21  23  25  28  30  33  35  38  …  307  307  316  322  326  329  356
 4   5   4   5   6   4   5   5   6   6      11    9   11   10   13   11   11
 0   0   1   1   0   1   1   0   0   1       6    8    7    8    5    7    8

In [50]:
function cheapest_win(perms)
    for gear in eachcol(perms)
        if works(gear[2], gear[3])[1]
            return gear[1]
        end
    end
    return Inf
end
cheapest_win(perms)

111

In [49]:
function expensive_loss(perms)
    for gear in eachcol(perms[1:end, end:-1:1])
        if !works(gear[2], gear[3])[1]
            return gear[1]
        end
    end
    return 0
end
expensive_loss(perms)

188